# Inference on images using PyTorch model

In [2]:
import os
import torch
import cv2
from utils.feature_extractor import featureExtractor
from utils.data_loader import TestDataset
from torch.utils.data import DataLoader
import numpy as np
# import pandas as pd
from time import time
import onnxruntime as ort


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

/home/yeldar/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def run_testing(trained_model, dataset_dir):
    img_list = os.listdir(dataset_dir)
    tp, tn, fp, fn = 0, 0, 0, 0
    names = []
    scores = []
    labels = []
    # st = time()
    for ind, image_name in enumerate(img_list):

        # try:
        print("Blurry Image Prediction: %d / %d images processed.." % (ind, len(img_list)))

        # Read the image
        img = cv2.imread(os.path.join(dataset_dir, image_name), 0)
        prediction, score = is_image_blurry(trained_model, img, threshold=0.5)
        # print(prediction, score)
        label = int(image_name.split('.')[0][-1])
        if (prediction):
            if (label):
                fp += 1
            else:
                tp += 1
        else:
            if (label):
                tn += 1
            else:
                fn += 1
        names.append(image_name)
        scores.append(score)
        labels.append(label)
        # except:
        #     continue

    # df = pd.DataFrame({"Name": names, "Score": scores, "Labels": labels})
    # e = time()
    # print("SPENT Time", e-st)
    print(tp, fp, tn, fn)
    p = tp/(tp+fp)
    r = tp/(tp+fn)
    f1 = 2*p*r/(p+r)
    acc = (tp+tn)/(tp+tn+fn+fp)
    print("Precision: ", p, "Recall: ", r, "F1-score: ", f1, "Accuracy: ", acc)
    # return df


def is_image_blurry(trained_model, img, threshold=0.5):
    feature_extractor = featureExtractor()
    accumulator = []

    # Resize the image by the downsampling factor
    feature_extractor.resize_image(img, np.shape(img)[0], np.shape(img)[1])

    # compute the image ROI using local entropy filter
    feature_extractor.compute_roi()

    # extract the blur features using DCT transform coefficients
    extracted_features = feature_extractor.extract_feature()
    extracted_features = np.array(extracted_features)

    if(len(extracted_features) == 0):
        return True
    test_data_loader = DataLoader(TestDataset(extracted_features), batch_size=1, shuffle=False)
    # ort_sess = ort.InferenceSession('blur.onnx')
    for batch_num, input_data in enumerate(test_data_loader):
        x = input_data
        x = x.to(device).float()
        output = trained_model(x)
        # output = ort_sess.run(None, {'input': x.numpy()})
        _, predicted_label = torch.max(output, 1)
        accumulator.append(predicted_label.item())
    
    prediction= np.mean(accumulator) < threshold
    return prediction, np.mean(accumulator)

In [4]:
trained_model = torch.load('./trained_model/trained_model-Kaggle_dataset1')
trained_model = trained_model['model_state'].to(device)
# trained_model = onnx.load("blur.onnx")
# dataset_dir = '/home/yeldar/Documents/bluriness/data/test2'
# run_testing(trained_model, dataset_dir)


#  Inference on single image using ONNX model

In [1]:
def predict(session, img): 
    #Run inference using ONNX model
    #Parameters:
    #    session (onnxruntime.InferenceSession): inference session based on provided onnx model 
    #    img (str): path to image 

    #Returns:
    #    prediction (bool): True if image blurred, False otherwise
    #    score (float): prediction score returned by model 

    feature_extractor = featureExtractor()
    accumulator = []
    threshold=0.5
    feature_extractor.resize_image(img, np.shape(img)[0], np.shape(img)[1])

    # compute the image ROI using local entropy filter
    feature_extractor.compute_roi()

    # # extract the blur features using DCT transform coefficients
    extracted_features = feature_extractor.extract_feature()
    extracted_features = np.array(extracted_features)

    if(len(extracted_features) == 0):
        print("Error processing features")

    test_data_loader = DataLoader(TestDataset(extracted_features), batch_size=1, shuffle=False)

    for batch_num, input_data in enumerate(test_data_loader):
        x = input_data
        x = x.float()
        x = np.resize(x,(1,x.shape[1]))
        output = session.run(['output'], {'input': x})
        output = torch.tensor(output[0])
        _, predicted_label = torch.max(output, 1)
        accumulator.append(predicted_label.item())
    
    score = np.mean(accumulator)
    prediction_label = score < threshold
    return prediction_label, score

In [4]:
img = cv2.imread("0_1.jpg", 0)
ort_sess = ort.InferenceSession('blur.onnx', None)
pred, score = predict(ort_sess, img)
print(pred, score)

False 0.7723577235772358


# Exporting model to ONNX

In [8]:
#Function to Convert to ONNX 
def Convert_ONNX(model): 

    # set the model to inference mode 
    model.eval() 

    dummy_input = torch.randn(1, 528, requires_grad=False).to(device)
    onnx_name = "blur.onnx"
    torch.onnx.export(model, dummy_input, onnx_name, verbose=True,
                  input_names = ['input'],   # the model's input names
                  output_names = ['output']) # the model's output names
    
    # export_output.save(onnx_name)
    print(" ") 
    print('Model has been converted to ONNX')

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
Convert_ONNX(trained_model)

Linear(in_features=528, out_features=64, bias=True)
Exported graph: graph(%input : Float(1, 528, strides=[528, 1], requires_grad=0, device=cuda:0),
      %fc1.weight : Float(64, 528, strides=[528, 1], requires_grad=1, device=cuda:0),
      %fc1.bias : Float(64, strides=[1], requires_grad=1, device=cuda:0),
      %fc2.weight : Float(32, 64, strides=[64, 1], requires_grad=1, device=cuda:0),
      %fc2.bias : Float(32, strides=[1], requires_grad=1, device=cuda:0),
      %fc3.weight : Float(2, 32, strides=[32, 1], requires_grad=1, device=cuda:0),
      %fc3.bias : Float(2, strides=[1], requires_grad=1, device=cuda:0)):
  %/fc1/Gemm_output_0 : Float(1, 64, strides=[64, 1], requires_grad=1, device=cuda:0) = onnx::Gemm[alpha=1., beta=1., transB=1, onnx_name="/fc1/Gemm"](%input, %fc1.weight, %fc1.bias), scope: utils.MLP.MLP::/torch.nn.modules.linear.Linear::fc1 # /home/yeldar/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/modules/linear.py:114:0
  %/Relu_output_0 : Float(1, 64, strid